# NLU+ 2021-2022: Lab 2 - Distributed Representations
#### Authors: Christos Baziotis, Lexi Birch, Frank Keller

In this lab, you will be working with pretrained
word embeddings using NumPy. Using a small text classification dataset and pretrained word embeddings,
you will do simple vector arithmetic operations, produce text representations for each example in the text data and do simple analysis and visualizations.

##### Time Management Notes
 - The lab is long. If you get stuck in a question please ask for help from the demonstrators.
 - You don't have to answer all the questions, you may skip a question if you are running out of time and work on them later on your own.

The snippet below contains some package imports and helper functions.

In [1]:
import csv
from typing import List
import numpy as np

np.set_printoptions(
    suppress=True)  # suppresses the use of scientific notation for small numbers


# you may use this function to print a numpy array and its properties
def print_array(arr):
    print(arr)
    print("shape:", arr.shape)
    print("type:", arr.dtype.type)
    print()

### Pretrained Word Embeddings

First, we will load the 50d GloVe pretrained word embeddings (https://nlp.stanford.edu/projects/glove/).

 - `embeddings` is the 2D matrix that holds all the word embeddings
 - `word2id` is a dictionary mapping each word to the index (row) it occurs in the `embeddings` matrix
 - `id2word` is a dictionary mapping each index (of the `embeddings` matrix) to the corresponding word

In [2]:
def load_glove(file):
    np.random.seed(0)

    _word2id = {}
    _id2word = {}
    _embs = []
    print("Loading embeddings...")
    with open(file, encoding='utf8') as f:
        for i, line in enumerate(f):
            values = line.split()
            word = values[0]
            _word2id[word] = i
            _id2word[i] = word
            _embs.append(np.asarray(values[1:], dtype='float32'))

    # <PAD> - this token is used for artifically increasing the length of a sentence
    _embs.append(np.random.normal(loc=0, scale=0.0005, size=(len(values[1:]),)))
    _word2id["<PAD>"] = i + 1
    _id2word[i + 1] = "<PAD>"

    # <UNK> - this token is used when a word does not exist in our vocabulary
    _embs.append(np.random.normal(loc=0, scale=0.05, size=(len(values[1:]),)))
    _word2id["<UNK>"] = i + 2
    _id2word[i + 2] = "<UNK>"

    _embs = np.array(_embs)
    print("Done!")
    print_array(_embs)
    return _embs, _word2id, _id2word


EMBEDDING_PATH = 'glove.6B.50d.txt'
embeddings, word2id, id2word = load_glove(EMBEDDING_PATH)

Loading embeddings...
Done!
[[ 0.41800001  0.24968    -0.41242    ... -0.18411    -0.11514
  -0.78580999]
 [ 0.013441    0.23682    -0.16899    ... -0.56656998  0.044691
   0.30392   ]
 [ 0.15164     0.30177    -0.16763    ... -0.35652     0.016413
   0.10216   ]
 ...
 [ 0.072617   -0.51393002  0.47279999 ... -0.18907    -0.59021002
   0.55558997]
 [ 0.00088203  0.00020008  0.00048937 ...  0.00038875 -0.00080695
  -0.00010637]
 [-0.04477333  0.01934512 -0.02554026 ...  0.08929352  0.0063456
   0.02009947]]
shape: (400002, 50)
type: <class 'numpy.float64'>



In [3]:

print(word2id["hello"])
print(id2word[85])

13075
world


### Question 1. Word Vector Similarity

Similar words are located near to each other in the embedding space.

**Question 1.1:** Implement a function, that given a word, it will retrieve its word embedding, find its `N` closest word embeddings to it, and finally print the corresponding words.
Use the dot product (i.e., `np.dot`) as the distance metric. The function should *not* return the original (input) word.

In [ ]:
def most_similar_dot(w: str, top: int = 10):
    emb_str = embeddings[word2id[w]]
    distance = {}
    for i,row in enumerate(embeddings):
        if row != emb_str:
            if len(topN) < 10:
                topN[np.dot(emb_str, row)] = id2word[i]
            else:
                if topN[]
                

for word in ["scotland", "edinburgh", "university", "informatics", "neural"]:
    print("\n","-"*10, word, "-"*10)
    print(most_similar_dot(word, top=10))

**Question 1.2:** Update the function to use the cosine similarity as metric, which is defined as:
$$
\cos(u,v)=\frac{u\cdot v}{\left\|u\right\|\left\|v\right\|}
$$

In [ ]:
def most_similar_cosine(w: str, top: int = 10):
    # --------------------------------------------------
    # Write your solution here
    # --------------------------------------------------


for word in ["scotland", "edinburgh", "university", "informatics", "neural"]:
    print("\n","-"*10, word, "-"*10)
    print(most_similar_cosine(word, top=10))

While both metrics mostly return similar results, there are some differences. Cosine similarity should be less noisy than the dot product. Think about why this happens.

##### Answer
The cosine is independent of vector length. It has been shown that the norm (i.e., length) of vectors encode information related to word frequency and not semantic or syntactic information.

### Question 2. Word Analogies

Word embeddings capture semantic and syntactic information. This is reflected in how they are organized in the embedding space.
Specifically, a nice property of word embeddings is that _sometimes_ they exhibit the ability to solve analogies, using simple vector arithmetic operations.

In the word analogy task, we are trying to find the word (embedding) that completes (solves) the following sentence (equation)  "*word_1* is to *word_2*, as *word_3* is to **???**". A famous example is "*man* is to *woman*, as *king* is to **queen**".

Mathematically, this is formulated as:


\begin{align}
&\vec{woman} - \vec{man} \approx \vec{queen} - \vec{king}  \\
\Rightarrow & \vec{king} - \vec{man} + \vec{woman} \approx \vec{queen}
\end{align}


![Word2Vec Analogies](https://developers.google.com/machine-learning/crash-course/images/linear-relationships.svg)
image source: https://developers.google.com/machine-learning/crash-course


**Question 2.1** Implement the `word_analogy()` function, which will accept a positive and a negative word list and will form a query vector out of these. The function should return the most similar words to the query vector. The list of negative words can be empty. Use the cosine similarity as a metric.

In [ ]:
def word_analogy(positive: List[str] = [], negative: List[str] = [],
                 top: int = 3):
    """
    This function implements the word analogy task (e.g., king-man+woman=?).
    Args:
        positive: list of words with positive sign
        negative: list of words with negative sign
        top: the N most similar words to return

    Returns:
        List of the top N most similar words
    """
    # --------------------------------------------------
    # Write your solution here
    # --------------------------------------------------


# First, let's check the result of the analogy in the example above
print(word_analogy(positive=["king", "woman"], negative=["man"])[0])

In [ ]:
# Next, let's see how the result(s) vary as we change the input query
print(word_analogy(positive=["king"]))
print(word_analogy(positive=["king", "woman"]))
print(word_analogy(positive=["king", "woman"], negative=["man"]))

**Question 2.2** Using the `word_analogy()` function and the equation in Question-2 description, answer the analogy questions below.

The result should show that the embeddings produce meaningful answers for both sematic and syntactic analogies. However, they don't get everything right. Can you think of some reasons?

Feel free to add/test more analogies!

In [ ]:
questions = [
    # semantic analogies
    ('greece', 'athens', 'iraq'),
    ('italy', 'italian', 'spain'),
    ('india', 'delhi', 'japan'),
    ('man', 'woman', 'boy'),

    # grammatical analogies
    ('small', 'smaller', 'large'),
    ('happy', 'happiest', 'sad'),
    ('scotland', 'scottish', 'greece'),
    ('feeding', 'fed', 'sitting'),
    ('good', 'awesome', 'bad'),
    # the results below are not as expected
    ('good', 'perfect', 'bad'),
    ('scotland', 'edinburgh', 'england'),
]

for q in questions:
    # --------------------------------------------------
    # Write your solution here
    # --------------------------------------------------

    # word1 is to word2, as word3 is to ?
    result = ...

    print('"{}" -> "{}", as "{}" -> "{}"'.format(*q, result))

##### Reasons why analogies can "fail"

 - The embeddings were not trained on sufficient data to learn good representations, in order to be able to solve that particular analogy
 - In the example we used small embeddings with only 50 dimensions, which prevents them from encoding fine-grained information

Also, some of the analogies may actually make sense, even if unexpected. For example:
`"scotland" -> "edinburgh", as "england" -> "oxford"` makes sense if the analogy is "most important university" instead of "capital". Therefore, the subtraction produces *an* analogy, but not necessarily the one we expected.

##### 2.3 Polysemous words (Go to Q3 if running our of time)
Polysemous words ([wiki](https://en.wikipedia.org/wiki/Polysemy)) have more than one meaning. Such an example is the word `java`, which can has three meanings:
 - The island in Indonesian
 - A type of coffee bean
 - A programming language

Using this word as an example, and with the help of the `word_analogy` function, try to isolate, or bring out, each of these meanings.

Also, explore other polysemous words.

In [ ]:
# let's see how we can isolate the meaning of polysemous words
print("java=", word_analogy(positive=["java"], top=50), "\n")


# --------------------------------------------------
# Write your solution here
# --------------------------------------------------

In [ ]:
# Try another polysemous word

# --------------------------------------------------
# Write your solution here
# --------------------------------------------------


**Optional**: Can you try and solve an analogy (like in Question 2.2) with one of these polysemous words? Do the results make sense?

In [ ]:
# word analogy with polysemous words
questions = [
    ('bird', 'pigeon', 'dog'),  # this should make sense
    ('snake', 'python', 'dog'),  # this should not make sense
]
for q in questions:
    result = word_analogy(positive=[q[1], q[2]], negative=[q[0]])[0]
    print('"{}" -> "{}", as "{}" -> "{}"'.format(*q, result))

### Text Corpus Datasets

Next, we will use the DBpedia ontology text classification dataset. The data are extracted from Wikipedia articles organized into different classes. It contains 560,000 training samples and 70,000 testing samples for each of 14 non-overlapping classes from DBpedia 2014. We are going to use a subset of the test data for the (analysis) purposes of this lab.

The classes are:

- Company

- Educational Institution

- Artist

- Athlete

- OfficeHolder

- Mean Of Transportation

- Building

- Natural Place

- Village

- Animal

- Plant

- Album

- Film

- Written Work

The data are stored in a csv file, and each row (record) is structured as follows:

 - class: denoting the class (1-14)

 - title: title of the article

 - description: the contents of the article

__Source__
https://papers.nips.cc/paper/5782-character-level-convolutional-networks-for-text-classification.pdf
https://wiki.dbpedia.org/
https://github.com/le-scientifique/torchDatasets/raw/master/dbpedia_csv.tar.gz


In [ ]:
from nltk.tokenize import word_tokenize

# let's load the text data
with open('dbpedia_csv/test.csv', 'r') as f:
    data = list(csv.reader(f, delimiter=","))
data = np.array(data)
classes = data[:, 0].astype(int)
titles = [list(word_tokenize(x.lower())) for x in data[:, 1]]
texts = [list(word_tokenize(x.lower())) for x in data[:, 2]]

In [ ]:
# Let's inspect the contents of each variable
print("classes:", classes[:5], "\n")
print("titles:", titles[:5], "\n")
print("texts:", texts[:5], "\n")

### 3. Document/Sentence Representations

Using the pretrained embeddings you will compute different text representations for a random batch of sentences. To start with, you can use the `batchify_sentences()` function below, to convert a batch of text sentences into a batch with word-ids. You can uncomment the printing commands to inspect what each step of the function does.

In [ ]:
from itertools import zip_longest

# create a batch of random sentences
def batchify_sentences(sents):
    """
    Transforms a list of sentences (word lists) in a 2D batch,
    by mapping each word into its word-id (i.e., row in embeddings matrix)
    """
    # 1 - map words to ids
    _ids = [[word2id.get(w, word2id["<UNK>"]) for w in s] for s in sents]
    # for x in _ids:
    #     print(x)

    # 2 - apply zero padding to make the sentences in the batch have the same length
    _ids = np.array(list(zip_longest(*_ids, fillvalue=word2id["<PAD>"]))).T
    # print_array(_ids)

    # 3 - construct some helper variables
    _lengths = np.array([len(x) for x in sents])
    # print_array(lengths)
    _mask = np.array(_ids != word2id["<PAD>"]).astype(int)
    # print_array(mask)

    return _ids, _lengths, _mask

Now, let's sample some sentences from the text corpus and transform them into a batch of ids. The `batchify_sentences` function returns:

 - A 2D `int` array with the word ids of each text sample, padded with `<PAD>` tokens, based on the length of the longest sentence in each batch.
 - A 1D `int` array (vector) with the **real** length (in words) of each text sample.
 - A 2D `boolean` array indicating whether each word is padded or not.

In [ ]:
BATCH_SIZE = 5
np.random.seed(1)  # for reproducibility
ids = set(np.random.choice(len(texts), BATCH_SIZE, replace=False))
batch = [texts[i] for i in ids]
# for x in batch:
#     print(x)
batch_ids, batch_lengths, batch_mask = batchify_sentences(batch)

In [ ]:
# Let's inspect what is in each variable
print("batch_ids", batch_ids.shape, '\n', batch_ids)
print("batch_lengths", batch_lengths.shape, '\n', batch_lengths)
print("batch_mask", batch_mask.shape, '\n', batch_mask)

#### 3.1 Vectorize the batch

Convert the 2D `batch_ids` array into a 3D tensor, by replacing each word id with the corresponding word embedding.

Do not use for loops. Each operation is independent therefore they can be run in parallel.

Think about mapping a function onto each element of the `batch` array.

The expected shape is `(5, 78, 50)`


In [ ]:
# vectorize the batch
def vectorize_batch(batch_idx):
    # --------------------------------------------------
    # Write your solution here
    # --------------------------------------------------

    return ...


batch_vec = vectorize_batch(batch_ids)
print_array(batch_vec)

#### 3.2 Max-Pooling
Compute sentence representations using max-pooling (i.e., maximum over each embedding dimension). The output shape should be `(5, 50)`.

In [ ]:
# --------------------------------------------------
# Write your solution here
# --------------------------------------------------
max_pooling = ...
print_array(max_pooling)

#### 3.3 Mean-Pooling (Average-Pooling)
Compute sentence representations using mean-pooling. This means computing the centroid of each sentence (average of its word embeddings).

Do not use for loops.

Each sentence has different length! Take that into account when computing the average. You should use `batch_lengths` and `batch_mask`.

The output shape should be `(5, 50)`.


In [ ]:
def mean_pooling(inputs, lengths, masks):
    # --------------------------------------------------
    # Write your solution here
    # --------------------------------------------------
    out = ...

    return out


print_array(mean_pooling(batch_vec, batch_lengths, batch_mask))


### 4. Inspection of Document Embeddings
In the previous question, you implemented a function that creates a single vector representation (i.e., embedding) out of its word embeddings. In this question you will use these text representations to inspect what information they encode.

Note that, since we construct text embeddings using simple arithmetic operations out of their constituent word embedding, then the text embeddings themselves will ``live'' in the same embedding space, and as a result, it will allow us to make comparisons between them.

**Question 4.1** Create a single vector representation for each text sample in the data.
Construct a 2D `text_embeddings` array with shape `(70000, 50)`.

In [ ]:
text_embeddings = []
batch_size = 128

for i in range(0, len(texts), batch_size):
    batch = texts[i:i + batch_size]
    # --------------------------------------------------
    # Write your solution here
    # --------------------------------------------------

text_embeddings = np.concatenate(text_embeddings, axis=0)
print_array(text_embeddings)

**Question 4.2** We can create a "concept" vector out of a list of words, using its word embeddings. For example the list of words `["tomato", "pasta", "olive"]` can produce a concept vector related to food. We can construct a vector representation for a list of words in exactly the same way as for regular sentences.

Find the most similar sentences to a concept, defined as a list of words. You need to construct a vector representation out of the words of the concept and compare it with the vector representations of each text example in the data. You may choose any method you want for producing a single vector representation out of a a sequence of embeddings. In the snippet below, we already provide 3 concepts, but you may add more if you want.

In [ ]:
concepts = [
    ("edinburgh", "scotland"),
    ("tv", "cinema", "movie", "entertainment"),
    ("tomato", "pasta", "olive"),
]

for concept in concepts:
    print("\n","-"*10, concept, "-"*10)
    # --------------------------------------------------
    # Write your solution here
    # --------------------------------------------------
    
    ...
    
    # return the ids of the most similar words, excluding the input
    ids = ...
    for i in ids[:5]:
        print("[{}]".format(i), data[i][2])

**Question 4.3:** Construct a single representation for each *class* in the dataset and then print the `50` most representative words of that class.

- Think what is the best way to construct the class representation?
- If the results that you get contain a lot of generic (mostly function) words, what can you do to improve the results? Think about what you learnt about the polysemous words.

In [ ]:
classes = np.array(classes)
class_labels = [x.strip() for x in open('dbpedia_csv/classes.txt', 'r').readlines()]

for cls, label in enumerate(class_labels, 1):
    print("\n","-"*10, cls, label, "-"*10)

    # --------------------------------------------------
    # Write your solution here
    # --------------------------------------------------
    words = ...
    print(words)


### 5. Visualization

A usual step of an exploratory analysis is to visualize the data to gain insights. Here, you will create a simple plot with the purpose of inspecting how the document (i.e., the content of each article) representations are dispersed in the embedding space, according to their classes.

To do this, you will have to use a dimensionality reduction technique, in order to project the document representations to 2 dimensions, and to be able to plot them. While dimensionality reduction techniques results in loss of information, they can be use as tool to build intuition. Different dimensionality reduction techniques have different pros and cons.

Here, we will use `UMAP`, which is a fast non-linear dimensionality reduction algorithm. The code below computes the 2D document representations. For details in tweaking UMAP, see https://umap-learn.readthedocs.io/en/latest/parameters.html

##### References
 - McInnes, L, Healy, J, UMAP: Uniform Manifold Approximation and Projection for Dimension Reduction, ArXiv e-prints 1802.03426, 2018

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import umap
%matplotlib inline

np.random.seed(2022)

def project_2d(original_embeddings, n_neighbors=50, min_dist=0.25, metric='euclidean'):
    fit = umap.UMAP(
        n_neighbors=n_neighbors,
        min_dist=min_dist,
        n_components=2,
        metric=metric
    )
    return fit.fit_transform(original_embeddings)

text_embeddings_2d = project_2d(text_embeddings)


**Question 5.1** Create a scatter plot using document embeddings and color-code each point based on its class. To avoid clutter (i.e., many points on top of each other) and make the plot more presentable, plot only a subset of points per class (e.g., 500).

(Double-click to zoom in the plot)

In [ ]:
%matplotlib inline
plt.rcParams["figure.dpi"] = 300

fig, ax = plt.subplots(figsize=(7, 7))
# --------------------------------------------------
# Write your solution here
# --------------------------------------------------

# use different markers to differentiate similar colors
markers = ["*",  ">", ".", ] 
